# Análisis Exploratorio de Datos (EDA) - Salud Mental y Tasas de Suicidio

## 1. Carga y Limpieza de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualización
sns.set(style="whitegrid")

In [ ]:
# Cargar los datasets
suicide_rates_df = pd.read_csv('../Proyecto_mental/crude suicide rates.csv')
substance_use_df = pd.read_csv('../Proyecto_mental/share-with-alcohol-and-substance-use-disorders 1990-2016.csv')

### 1.1. Exploración Inicial del Dataset de Tasas de Suicidio

In [ ]:
print("Información del dataset de tasas de suicidio:")
suicide_rates_df.info()
print("
Primeras filas:")
display(suicide_rates_df.head())

### 1.2. Exploración Inicial del Dataset de Abuso de Sustancias

In [ ]:
print("Información del dataset de abuso de sustancias:")
substance_use_df.info()
print("
Primeras filas:")
display(substance_use_df.head())

### 1.3. Limpieza de Nombres de Columnas
Los nombres de las columnas son largos, contienen caracteres especiales y espacios. Vamos a limpiarlos para que sean más fáciles de usar.

In [ ]:
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = col.replace(' (%)', '_percent') \n                     .replace(' (per 100 000 population)', '_per_100k') \n                     .replace('[', '') \n                     .replace(']', '') \n                     .replace(':', '') \n                     .replace('-', '_') \n                     .replace(' ', '_') \n                     .lower()
        new_cols.append(new_col)
    df.columns = new_cols
    return df

# Limpiar nombres de columnas en ambos dataframes
suicide_rates_df_cleaned = clean_col_names(suicide_rates_df.copy())
substance_use_df_cleaned = clean_col_names(substance_use_df.copy())

# Renombrar columnas específicas para mayor claridad
suicide_rates_df_cleaned.rename(columns={'':'country', 'unnamed:_1': 'year'}, inplace=True)
substance_use_df_cleaned.rename(columns={'entity': 'country', 'prevalence___alcohol_and_substance_use_disorders_both_age_standardized_percent': 'substance_use_disorder_percent'}, inplace=True)

print("Columnas limpiadas del dataset de suicidios:")
print(suicide_rates_df_cleaned.columns)

print("
Columnas limpiadas del dataset de abuso de sustancias:")
print(substance_use_df_cleaned.columns)

El primer dataframe tiene una fila de encabezado adicional que se ha leído como la primera fila de datos. Necesitamos corregir esto.

In [ ]:
# La primera fila es en realidad parte del encabezado, vamos a omitirla y renombrar las columnas de nuevo
suicide_rates_df = pd.read_csv('../Proyecto_mental/crude suicide rates.csv', header=[0, 1])
suicide_rates_df.columns = ['_'.join(col).strip() for col in suicide_rates_df.columns.values]
suicide_rates_df = clean_col_names(suicide_rates_df)
suicide_rates_df.rename(columns={'unnamed:_0_level_0_country': 'country', 'unnamed:_1_level_0_year': 'year'}, inplace=True)

print("Columnas corregidas y limpiadas del dataset de suicidios:")
print(suicide_rates_df.columns)
display(suicide_rates_df.head())

### 1.4. Verificación de Valores Nulos

In [ ]:
print("Valores nulos en el dataset de suicidios:")
print(suicide_rates_df.isnull().sum())

print("
Valores nulos en el dataset de abuso de sustancias:")
print(substance_use_df_cleaned.isnull().sum())

El dataset de abuso de sustancias tiene valores nulos en la columna `code`. Dado que tenemos el nombre del país (`country`), podemos ignorar o eliminar la columna `code` si no es necesaria para la unión.

In [ ]:
# Eliminar la columna 'code' con valores nulos
substance_use_df_cleaned = substance_use_df_cleaned.drop(columns=['code'])

# Convertir las columnas de año a tipo numérico para la unión
suicide_rates_df['year'] = pd.to_numeric(suicide_rates_df['year'], errors='coerce')
substance_use_df_cleaned['year'] = pd.to_numeric(substance_use_df_cleaned['year'], errors='coerce')

# Eliminar filas donde el año pueda ser nulo después de la conversión
suicide_rates_df.dropna(subset=['year'], inplace=True)
substance_use_df_cleaned.dropna(subset=['year'], inplace=True)

# Convertir año a entero
suicide_rates_df['year'] = suicide_rates_df['year'].astype(int)
substance_use_df_cleaned['year'] = substance_use_df_cleaned['year'].astype(int)

### 1.5. Unificación y Fusión de los Datasets
Ahora, fusionaremos los dos dataframes usando `country` y `year` como claves.

In [ ]:
merged_df = pd.merge(suicide_rates_df, substance_use_df_cleaned, on=['country', 'year'], how='inner')

print("Información del dataset fusionado:")
merged_df.info()

print("
Primeras filas del dataset fusionado:")
display(merged_df.head())